In [5]:
import mysql.connector
from mysql.connector import Error
import os

In [6]:
# Database Connection
class Database:
    def __init__(self, host, user, password, database=None):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = None
        self.cursor = None
        self.connect()
    
    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            self.cursor = self.connection.cursor()
        except Error as e:
            print(e)

    def create_database(self, name):
        try:
            sql = f"CREATE DATABASE IF NOT EXISTS {name}"
            self.cursor.execute(sql)
            self.connection.database = name
            self.database = name
            print("Database Created!")
        except Error as e:
            print(e)

    def create_table(self, table_sql):
        try:
            self.cursor.execute(table_sql)
            print(f"{table_sql}: created")
        except Error as e:
            print(e)

    def execute_query(self, query, params=None):
        try:
            self.cursor.execute(query, params or ())
            if self.cursor.with_rows:
                return self.cursor.fetchall()
        except Error as e:
            print(e)

    def commit(self):
        self.connection.commit()
    
    def close(self):
        self.connection.close()

In [7]:
# User Management
class User:
    def __init__(self, db):
        self.db = db
    
    def register(self, username, password, email):
        query = "INSERT INTO users (username, password, email) VALUES (%s, %s, %s)"
        self.db.execute_query(query, (username, password, email))
        self.db.commit()


In [18]:
# Image Categories
class Category:
    def __init__(self, db):
        self.db = db
    
    def add_category(self, name):
        query = "INSERT INTO categories (name) VALUES (%s)"
        self.db.execute_query(query, (name,))
        self.db.commit()

    def get_all_categories(self):
        query = "SELECT * FROM categories"
        return self.db.execute_query(query)



In [30]:
# Image Handling
class Image:
    def __init__(self, db):
        self.db = db
    
    def upload_img(self, user_id, cat_id, title, desc, file_path):
        query = """INSERT INTO images
                (user_id, category_id, title, description, file_path)
                VALUES (%s, %s, %s, %s, %s)"""
        self.db.execute_query(query, (user_id, cat_id, title, desc, file_path))
        self.db.commit()

    def search_by_user(self, user_id):
        query = "SELECT * FROM images WHERE user_id = %s"
        return self.db.execute_query(query, (user_id,))

    def delete_img(self, img_id):
        query = "SELECT * FROM images WHERE id = %s"
        img = self.db.execute_query(query, (img_id,))
        if img:
            query = "DELETE FROM images WHERE id = %s"
            self.db.execute_query(query, (img_id,))
            self.db.commit()
            os.remove(img[0]['file_path'])


In [10]:
# Define tables
tables = {
    'users': """
        CREATE TABLE IF NOT EXISTS users (
            id INT AUTO_INCREMENT,
            username VARCHAR(50) NOT NULL,
            password VARCHAR(255) NOT NULL,
            email VARCHAR(100),
            PRIMARY KEY (id)
        )
    """,
    'categories': """
        CREATE TABLE IF NOT EXISTS categories (
            id INT AUTO_INCREMENT,
            name VARCHAR(100) NOT NULL,
            PRIMARY KEY (id)
        )
    """,
    'images': """
        CREATE TABLE IF NOT EXISTS images (
            id INT AUTO_INCREMENT,
            user_id INT NOT NULL,
            category_id INT,
            title VARCHAR(255) NOT NULL,
            description TEXT,
            file_path VARCHAR(255) NOT NULL,
            PRIMARY KEY (id),
            FOREIGN KEY (user_id) REFERENCES users (id),
            FOREIGN KEY (category_id) REFERENCES categories (id)
        )
    """
}

In [11]:
db_info = {
    'host': 'localhost',
    'user': 'root',
    'password': 'pass'
}
db = Database(**db_info)


In [12]:
vars(db)

{'host': 'localhost',
 'user': 'root',
 'password': 'pass',
 'database': None,
 'connection': <mysql.connector.connection_cext.CMySQLConnection at 0x7fa9e106ff40>,
 'cursor': <mysql.connector.cursor_cext.CMySQLCursor at 0x7fa9f075e3a0>}

In [14]:
db.create_database("img_gallery")

Database Created!


In [15]:
vars(db)

{'host': 'localhost',
 'user': 'root',
 'password': 'pass',
 'database': 'img_gallery',
 'connection': <mysql.connector.connection_cext.CMySQLConnection at 0x7fa9e106ff40>,
 'cursor': <mysql.connector.cursor_cext.CMySQLCursor at 0x7fa9f075e3a0>}

In [13]:
for table in tables.values():
    db.create_table(table)

In [16]:
user1 = User(db)
user1.register("u1", "123", "a.b@gmail.com")


In [19]:
category1 = Category(db)
category1.add_category("c2")

In [20]:
data = category1.get_all_categories()
print(data)

[(1, 'c1'), (2, 'c2')]


In [31]:
img1 = Image(db)
img1.upload_img(1, 2, "t1", "d1", "Desktop")

In [32]:
img1.search_by_user(1)

[(1, 1, 1, 't1', 'd1', 'Desktop'),
 (2, 1, 2, 't1', 'd1', 'Desktop'),
 (3, 1, 2, 't1', 'd1', 'Desktop'),
 (4, 1, 2, 't1', 'd1', 'Desktop')]

In [33]:
img1.delete_img(3)

In [34]:
img1.search_by_user(1)

[(1, 1, 1, 't1', 'd1', 'Desktop'),
 (2, 1, 2, 't1', 'd1', 'Desktop'),
 (4, 1, 2, 't1', 'd1', 'Desktop')]